In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

url = "https://map.naver.com/p?c=15.00,0,0,0,dh"


options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
"""test
    """
driver.get(url)
time.sleep(1)

search_btn = driver.find_element(By.CSS_SELECTOR, "#home_search_input_box > div > div")
search_btn.click()

In [10]:
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

URL = "https://map.naver.com/p?c=15.00,0,0,0,dh"

# 리뷰 영역 래퍼(너가 준 구조 기반)
REVIEW_WRAP  = "#app-root > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.f7aZ0 > div.dAsGb"
REVIEW_LINKS = REVIEW_WRAP + " a"   # ✅ a만 모아서 '방문자리뷰/블로그리뷰' 텍스트로 구분 (배지 영향 없음)


def switch_default(driver):
    driver.switch_to.default_content()


def switch_to_iframe(driver, wait, iframe_id: str):
    switch_default(driver)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, iframe_id)))


def safe_text(driver, css: str) -> str:
    try:
        return driver.find_element(By.CSS_SELECTOR, css).text.strip()
    except:
        return ""


def to_int_count(text: str) -> int:
    """
    '방문자리뷰 1,234' / '블로그리뷰 56' / '1,234' 같은 문자열에서 숫자만 뽑아서 int로.
    못 뽑으면 0.
    """
    if not text:
        return 0
    m = re.search(r"([\d,]+)", text)
    if not m:
        return 0
    return int(m.group(1).replace(",", ""))


def extract_visit_blog_counts(driver) -> dict:
    """
    ✅ 새로오픈 배지(span) 유무로 nth-child가 밀려도 상관없이
    리뷰 링크(a)들만 모아서 텍스트에 '방문/블로그' 포함 여부로 각각 카운트 추출
    """
    visit_cnt = 0
    blog_cnt = 0

    links = driver.find_elements(By.CSS_SELECTOR, REVIEW_LINKS)
    for a in links:
        t = (a.text or "").strip()
        # 예: '방문자리뷰 123', '블로그리뷰 45'
        if "방문" in t and "리뷰" in t:
            visit_cnt = to_int_count(t)
        elif "블로그" in t and "리뷰" in t:
            blog_cnt = to_int_count(t)

    return {"visit_review_count": visit_cnt, "blog_review_count": blog_cnt}


def search_keyword(driver, wait, keyword: str):
    switch_default(driver)

    # 검색 버튼 클릭
    search_btn = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#home_search_input_box > div > div"))
    )
    search_btn.click()

    # 검색어 입력
    search_input = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input.input_search"))
    )
    search_input.click()
    search_input.send_keys(Keys.CONTROL, "a")
    search_input.send_keys(Keys.BACKSPACE)
    search_input.send_keys(keyword)
    search_input.send_keys(Keys.ENTER)


def load_all_results(driver, wait, max_rounds=100, sleep_each=0.7) -> int:
    """
    searchIframe에서 결과 리스트를 끝까지 스크롤해 전체 로딩을 끝냄(무한로딩 대응).
    """
    switch_to_iframe(driver, wait, "searchIframe")

    container = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#_pcmap_list_scroll_container"))
    )

    prev = 0
    stable = 0

    for _ in range(max_rounds):
        items = driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li")
        cur = len(items)

        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", container)
        time.sleep(sleep_each)

        items2 = driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li")
        cur2 = len(items2)

        if cur2 == prev:
            stable += 1
        else:
            stable = 0
            prev = cur2

        # 증가가 3회 연속 없으면 로딩 종료로 간주
        if stable >= 3:
            break

    return len(driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li"))


def click_nth_result(driver, wait, n: int):
    """
    searchIframe에서 n번째 결과 클릭
    """
    switch_to_iframe(driver, wait, "searchIframe")

    li_css = f"#_pcmap_list_scroll_container > ul > li:nth-child({n})"
    li = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, li_css)))

    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", li)
    time.sleep(0.15)

    try:
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, li_css))).click()
    except:
        driver.execute_script("arguments[0].click();", li)


def extract_detail(driver, wait) -> dict:
    """
    entryIframe(상세)에서:
    - 가게명
    - 방문자리뷰 수
    - 블로그리뷰 수
    추출
    """
    switch_to_iframe(driver, wait, "entryIframe")

    # 가게명 (후보 여러개)
    name = ""
    for sel in ["h1", "h2", "span.Fc1rA"]:
        t = safe_text(driver, sel)
        if t:
            name = t
            break

    counts = extract_visit_blog_counts(driver)

    return {
        "name": name,
        **counts
    }


def crawl_all(keyword: str, headless: bool = False) -> pd.DataFrame:
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    if headless:
        options.add_argument("--headless=new")

    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 15)

    driver.get(URL)

    search_keyword(driver, wait, keyword)

    total = load_all_results(driver, wait)
    print(f"✅ 결과 로딩 완료: {total}개 감지")

    rows = []
    for i in range(1, total + 1):
        try:
            click_nth_result(driver, wait, i)

            # 상세 iframe 로딩 안정화 (네이버 지도는 전환이 종종 느림)
            time.sleep(0.5)

            data = extract_detail(driver, wait)
            data["rank"] = i
            rows.append(data)

            print(f"[{i}/{total}] {data['name']} | 방문 {data['visit_review_count']} | 블로그 {data['blog_review_count']}")
        except Exception as e:
            print(f"[{i}/{total}] ❌ 실패: {e}")
            continue

    driver.quit()

    df = pd.DataFrame(rows, columns=["rank", "name", "visit_review_count", "blog_review_count"])
    return df


if __name__ == "__main__":
    kw = input("🔍 검색어 입력: ").strip()
    df = crawl_all(kw, headless=False)

    # 확인
    print(df.head(20))
    print(f"\n총 수집: {len(df)}건")


🔍 검색어 입력:  상봉 맛집


✅ 결과 로딩 완료: 42개 감지
[1/42] 꿀삼겹 본점 | 방문 2512 | 블로그 4586
[2/42] 천복집 흑염소 상봉1호점 | 방문 493 | 블로그 84
[3/42] 아웃백스테이크하우스 상봉홈플러스점 | 방문 1942 | 블로그 450
[4/42] 애슐리퀸즈 엔터식스 상봉점 | 방문 1290 | 블로그 251
[5/42] 쿠우쿠우 골드 상봉점 | 방문 7210 | 블로그 2368
[6/42] 카페252 | 방문 4569 | 블로그 630
[7/42]  | 방문 6377 | 블로그 952
[8/42]  | 방문 450 | 블로그 49
[9/42]  | 방문 650 | 블로그 73
[10/42]  | 방문 638 | 블로그 523
[11/42]  | 방문 638 | 블로그 523
[12/42]  | 방문 638 | 블로그 523
[13/42]  | 방문 89 | 블로그 37
[14/42]  | 방문 89 | 블로그 37
[15/42]  | 방문 89 | 블로그 37
[16/42]  | 방문 8386 | 블로그 235
[17/42]  | 방문 2512 | 블로그 4586
[18/42]  | 방문 2512 | 블로그 4586
[19/42]  | 방문 2512 | 블로그 4586
[20/42]  | 방문 2512 | 블로그 4586
[21/42]  | 방문 2512 | 블로그 4586
[22/42]  | 방문 2512 | 블로그 4586
[23/42]  | 방문 2512 | 블로그 4586
[24/42]  | 방문 2512 | 블로그 4586
[25/42]  | 방문 2512 | 블로그 4586
[26/42]  | 방문 2512 | 블로그 4586
[27/42]  | 방문 2512 | 블로그 4586
[28/42]  | 방문 2512 | 블로그 4586
[29/42]  | 방문 2512 | 블로그 4586
[30/42]  | 방문 2512 | 블로그 4586
[31/42]  | 방문 89 | 블로그 37
[32/42] 팁씨 다이닝 | 방문 147 | 블로그

In [2]:
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

URL = "https://map.naver.com/p?c=15.00,0,0,0,dh"

# 상세 페이지 리뷰 영역(너가 준 구조 기반)
REVIEW_WRAP  = "#app-root > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.f7aZ0 > div.dAsGb"
REVIEW_LINKS = REVIEW_WRAP + " a"  # ✅ a만 모으면 배지(span) 때문에 밀려도 안전

# 검색 결과 페이지 하단 네비게이션(너가 준 형태)
PAGINATION_ANCHORS = "#app-root > div > div.XUrfU > div.zRM9F > a"
PAGINATION_WRAP    = "#app-root > div > div.XUrfU > div.zRM9F"


def switch_default(driver):
    driver.switch_to.default_content()


def switch_to_iframe(driver, wait, iframe_id: str):
    switch_default(driver)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, iframe_id)))


def safe_text(driver, css: str) -> str:
    try:
        return driver.find_element(By.CSS_SELECTOR, css).text.strip()
    except:
        return ""


def to_int_count(text: str) -> int:
    if not text:
        return 0
    m = re.search(r"([\d,]+)", text)
    return int(m.group(1).replace(",", "")) if m else 0


def extract_visit_blog_counts(driver) -> dict:
    """
    ✅ 새로오픈 배지 유무로 nth-child가 밀려도 상관없이
    리뷰 링크(a)들 텍스트로 '방문/블로그'를 구분해 각각 수집
    """
    visit_cnt = 0
    blog_cnt = 0

    links = driver.find_elements(By.CSS_SELECTOR, REVIEW_LINKS)
    for a in links:
        t = (a.text or "").strip()
        if "방문" in t and "리뷰" in t:
            visit_cnt = to_int_count(t)
        elif "블로그" in t and "리뷰" in t:
            blog_cnt = to_int_count(t)

    return {"visit_review_count": visit_cnt, "blog_review_count": blog_cnt}


def search_keyword(driver, wait, keyword: str):
    switch_default(driver)

    search_btn = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#home_search_input_box > div > div"))
    )
    search_btn.click()

    search_input = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input.input_search"))
    )
    search_input.click()
    search_input.send_keys(Keys.CONTROL, "a")
    search_input.send_keys(Keys.BACKSPACE)
    search_input.send_keys(keyword)
    search_input.send_keys(Keys.ENTER)


def load_all_results_current_page(driver, wait, max_rounds=120, sleep_each=0.6) -> int:
    """
    현재 페이지(searchIframe)의 결과 리스트를 끝까지 스크롤하여 무한로딩 종료
    """
    switch_to_iframe(driver, wait, "searchIframe")

    container = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#_pcmap_list_scroll_container"))
    )

    prev = -1
    stable = 0

    for _ in range(max_rounds):
        items = driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li")
        cur = len(items)

        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", container)
        time.sleep(sleep_each)

        items2 = driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li")
        cur2 = len(items2)

        if cur2 == prev:
            stable += 1
        else:
            stable = 0
            prev = cur2

        # 3번 연속 증가 없으면 로딩 끝으로 간주
        if stable >= 3:
            break

    return len(driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li"))


def click_nth_result(driver, wait, n: int):
    """
    searchIframe에서 n번째 결과 클릭
    """
    switch_to_iframe(driver, wait, "searchIframe")

    li_css = f"#_pcmap_list_scroll_container > ul > li:nth-child({n})"
    li = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, li_css)))

    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", li)
    time.sleep(0.12)

    try:
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, li_css))).click()
    except:
        driver.execute_script("arguments[0].click();", li)


def extract_detail(driver, wait) -> dict:
    """
    entryIframe(상세)에서 가게명 + 방문/블로그리뷰 수 추출
    """
    switch_to_iframe(driver, wait, "entryIframe")

    # 가게명 후보
    name = ""
    for sel in ["h1", "h2", "span.Fc1rA"]:
        t = safe_text(driver, sel)
        if t:
            name = t
            break

    counts = extract_visit_blog_counts(driver)

    return {"name": name, **counts}


def get_next_pagination_button(driver, wait):
    """
    searchIframe에서 '다음 페이지'로 갈 버튼을 찾는다.
    - 너가 말한 a:nth-child(3), a:nth-child(4) 같은 고정 인덱스는 페이지마다 변동 가능해서
      ✅ 여기서는 "가장 오른쪽에 가까운 버튼들 중 클릭 가능한 것"을 우선으로 잡는다.
    - 더 확실히 하려면 aria-label / class 기반으로 잡는 게 좋은데, 지금은 DOM 변동 고려해 범용으로 구현.
    """
    switch_to_iframe(driver, wait, "searchIframe")

    # 네비게이션 영역이 아예 없으면 종료
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, PAGINATION_WRAP)))
    except TimeoutException:
        return None

    anchors = driver.find_elements(By.CSS_SELECTOR, PAGINATION_ANCHORS)
    if not anchors:
        return None

    # 뒤에서부터(오른쪽부터) "클릭 가능한 다음 후보" 탐색
    # - disabled/aria-disabled/클래스 등은 페이지마다 다를 수 있어, 일단 안전하게 '표시&활성' 위주
    for a in reversed(anchors):
        try:
            if not a.is_displayed():
                continue

            aria_disabled = (a.get_attribute("aria-disabled") or "").lower()
            if aria_disabled in ("true", "disabled"):
                continue

            cls = (a.get_attribute("class") or "").lower()
            if "disabled" in cls:
                continue

            # 현재 활성 페이지(선택됨) 버튼은 넘기고 싶을 수 있음
            # 다만 어떤 구조인지 확실치 않아 여기서는 "클릭 가능한 것" 우선
            return a
        except StaleElementReferenceException:
            continue

    return None


def click_next_page(driver, wait) -> bool:
    """
    다음 페이지로 이동 시도.
    성공하면 True, 더 이상 없으면 False
    """
    switch_to_iframe(driver, wait, "searchIframe")

    # 현재 페이지의 첫 결과 텍스트를 기록해서 페이지 전환을 감지
    first_before = ""
    try:
        first_before = driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li").text.strip()
    except:
        pass

    btn = get_next_pagination_button(driver, wait)
    if btn is None:
        return False

    # 클릭(일반 + JS fallback)
    try:
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
        time.sleep(0.1)
        btn.click()
    except:
        try:
            driver.execute_script("arguments[0].click();", btn)
        except:
            return False

    # 페이지가 바뀌었는지 대기: 첫 항목 텍스트 변화 또는 리스트 재로딩
    switch_to_iframe(driver, wait, "searchIframe")
    try:
        WebDriverWait(driver, 10).until(
            lambda d: (d.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li").text.strip() != first_before)
            if first_before else True
        )
    except:
        # 변화 감지가 실패해도, 다음 로딩 단계에서 판단 가능
        pass

    return True


def crawl_all_pages(keyword: str, headless: bool = False, max_pages: int = 999) -> pd.DataFrame:
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    if headless:
        options.add_argument("--headless=new")

    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 15)
    driver.get(URL)

    search_keyword(driver, wait, keyword)

    rows = []
    global_rank = 1
    page_no = 1

    while page_no <= max_pages:
        # 1) 현재 페이지 결과 로딩 끝까지
        total = load_all_results_current_page(driver, wait)
        print(f"\n📄 Page {page_no} | 로딩된 결과: {total}개")

        # 2) 현재 페이지 결과 전부 크롤링
        for i in range(1, total + 1):
            try:
                click_nth_result(driver, wait, i)
                time.sleep(0.45)

                data = extract_detail(driver, wait)
                data["page"] = page_no
                data["rank_in_page"] = i
                data["rank_global"] = global_rank
                rows.append(data)

                print(f"[P{page_no} {i}/{total}] {data['name']} | 방문 {data['visit_review_count']} | 블로그 {data['blog_review_count']}")
                global_rank += 1
            except Exception as e:
                print(f"[P{page_no} {i}/{total}] ❌ 실패: {e}")
                continue

        # 3) 다음 페이지 버튼 없으면 종료
        moved = click_next_page(driver, wait)
        if not moved:
            print("\n✅ 더 이상 다음 페이지가 없어 종료합니다.")
            break

        page_no += 1
        time.sleep(0.6)  # 페이지 전환 안정화

    driver.quit()

    df = pd.DataFrame(rows, columns=[
        "rank_global", "page", "rank_in_page",
        "name", "visit_review_count", "blog_review_count"
    ])
    return df


if __name__ == "__main__":
    kw = input("🔍 검색어 입력: ").strip()
    df = crawl_all_pages(kw, headless=False)
    print("\n--- 미리보기 ---")
    print(df.head(20))
    print(f"\n총 수집: {len(df)}건")


🔍 검색어 입력:  중랑구 면목동 일식



📄 Page 1 | 로딩된 결과: 50개
[P1 1/50] 카츠몬스터 면목점 | 방문 35 | 블로그 237
[P1 2/50] 만보카야 | 방문 298 | 블로그 210
[P1 3/50] 동경생포차 면목점 | 방문 163 | 블로그 215
[P1 4/50] 나베정 면목본점 | 방문 1083 | 블로그 198
[P1 5/50] 샤브온 사가정본점 | 방문 595 | 블로그 161
[P1 6/50] 대광어횟집 | 방문 694 | 블로그 261
[P1 7/50] 신선스시 | 방문 190 | 블로그 23
[P1 8/50] 금화왕돈까스 용마산역점 | 방문 2169 | 블로그 259
[P1 9/50] 다시 이자카야 면목점 | 방문 1441 | 블로그 209
[P1 10/50] 강태공의선술집 | 방문 835 | 블로그 65
[P1 11/50] ❌ 실패: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6091888d5
	0x7ff609188930
	0x7ff608f6165d
	0x7ff608fb9a33
	0x7ff608fb9d3c
	0x7ff60900df67
	0x7ff60900ac97
	0x7ff608faac29
	0x7ff608faba93
	0x7ff6094a0620
	0x7ff60949af60
	0x7ff6094b96c6
	0x7ff6091a5dd4
	0x7ff6091aed7c
	0x7ff609191ff4
	0x7ff6091921a5
	0x7ff609177ed2
	0x7ffdba15e8d7
	0x7ffdbaa0c53c

[P1 12/50] 면식당 서일대점 | 방문 408 | 블로그 10
[P1 13/50] AI 브리핑
실험 단계로 정확하지 않을 수 있어요.
안내 | 방문 20 | 블로그 9
[P1 14/50] 파파라멘 | 방문 341 | 블로그 140
[P1 15/50] 어촌 | 방문 103 | 블로그 27
[P1 16/50] 야키토리찬찬 | 방문 68 | 블로그 38
[

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6091888d5
	0x7ff609188930
	0x7ff608f61465
	0x7ff608fa9d83
	0x7ff608fe1ee2
	0x7ff608fdc500
	0x7ff608fdb6f0
	0x7ff608f2b325
	0x7ff6094a0620
	0x7ff60949af60
	0x7ff6094b96c6
	0x7ff6091a5dd4
	0x7ff6091aed7c
	0x7ff608f29cb8
	0x7ff6095fefe8
	0x7ffdba15e8d7
	0x7ffdbaa0c53c


In [3]:
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException


URL = "https://map.naver.com/p?c=15.00,0,0,0,dh"

# 상세 리뷰 영역
REVIEW_WRAP  = "#app-root > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.f7aZ0 > div.dAsGb"
REVIEW_LINKS = REVIEW_WRAP + " a"

# 검색 결과 페이지 하단 네비게이션
PAGINATION_ANCHORS = "#app-root > div > div.XUrfU > div.zRM9F > a"
PAGINATION_WRAP    = "#app-root > div > div.XUrfU > div.zRM9F"

# ✅ 주소 펼치기 + 주소 텍스트 (사용자가 준 셀렉터)
ADDR_TOGGLE_SVG = "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.tQY7D > div > a > span._UCia > svg"
ADDR_TEXT_SEL   = "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.tQY7D > div > div.Y31Sf"


# -------------------------
# 프레임/유틸
# -------------------------
def switch_default(driver):
    driver.switch_to.default_content()


def switch_to_iframe(driver, wait, iframe_id: str):
    switch_default(driver)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, iframe_id)))


def safe_text(driver, css: str) -> str:
    try:
        return driver.find_element(By.CSS_SELECTOR, css).text.strip()
    except:
        return ""


def to_int_count(text: str) -> int:
    if not text:
        return 0
    m = re.search(r"([\d,]+)", text)
    return int(m.group(1).replace(",", "")) if m else 0


def wait_presence(driver, wait, css: str, timeout=15):
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css))
    )


def wait_clickable(driver, wait, css: str, timeout=15):
    return WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css))
    )


def js_click(driver, el):
    driver.execute_script("arguments[0].click();", el)


# -------------------------
# 크롤링 로직
# -------------------------
def extract_visit_blog_counts(driver) -> dict:
    visit_cnt = 0
    blog_cnt = 0

    links = driver.find_elements(By.CSS_SELECTOR, REVIEW_LINKS)
    for a in links:
        t = (a.text or "").strip()
        if "방문" in t and "리뷰" in t:
            visit_cnt = to_int_count(t)
        elif "블로그" in t and "리뷰" in t:
            blog_cnt = to_int_count(t)

    return {"visit_review_count": visit_cnt, "blog_review_count": blog_cnt}


def extract_address(driver, wait) -> str:
    """
    ✅ 주소 아이콘(svg) 클릭 후, 주소 텍스트(Y31Sf) 추출.
    - 아이콘이 없거나 클릭이 필요없는 경우도 있으니 예외는 조용히 처리.
    """
    # 주소 텍스트가 이미 있으면 그대로 읽기
    addr = safe_text(driver, ADDR_TEXT_SEL)
    if addr:
        return addr

    # 아이콘이 있으면 클릭 시도
    try:
        toggle = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ADDR_TOGGLE_SVG))
        )
        # svg 자체가 클릭 안 먹는 경우가 있어 부모(a)를 클릭하도록 처리
        try:
            toggle.click()
        except:
            parent_a = toggle.find_element(By.XPATH, "./ancestor::a[1]")
            js_click(driver, parent_a)
    except:
        pass

    # 클릭 후 주소 텍스트 대기
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ADDR_TEXT_SEL))
        )
    except:
        pass

    return safe_text(driver, ADDR_TEXT_SEL)


def search_keyword(driver, wait, keyword: str):
    switch_default(driver)

    # 검색 버튼 클릭
    search_btn = wait_clickable(driver, wait, "#home_search_input_box > div > div", timeout=20)
    search_btn.click()

    # 검색어 입력
    search_input = wait_presence(driver, wait, "input.input_search", timeout=20)
    search_input.click()
    search_input.send_keys(Keys.CONTROL, "a")
    search_input.send_keys(Keys.BACKSPACE)
    search_input.send_keys(keyword)
    search_input.send_keys(Keys.ENTER)


def load_all_results_current_page(driver, wait, max_rounds=160, sleep_each=0.7) -> int:
    """
    현재 페이지(searchIframe)의 결과 리스트를 끝까지 스크롤하여 무한로딩 종료
    (너무 빠른 스크롤로 로딩이 씹히는 걸 막으려고 sleep_each를 살짝 늘림)
    """
    switch_to_iframe(driver, wait, "searchIframe")

    container = wait_presence(driver, wait, "#_pcmap_list_scroll_container", timeout=20)

    prev = -1
    stable = 0

    for _ in range(max_rounds):
        driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li")  # warm-up
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", container)
        time.sleep(sleep_each)

        cur2 = len(driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li"))

        if cur2 == prev:
            stable += 1
        else:
            stable = 0
            prev = cur2

        if stable >= 4:  # ✅ 3 -> 4로 늘려서 "진짜 끝"에 더 가깝게
            break

    return len(driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li"))


def click_nth_result(driver, wait, n: int):
    """
    searchIframe에서 n번째 결과 클릭 후,
    entryIframe이 올라올 때까지 확실히 기다림(멈춤 방지 핵심)
    """
    switch_to_iframe(driver, wait, "searchIframe")

    li_css = f"#_pcmap_list_scroll_container > ul > li:nth-child({n})"
    li = wait_presence(driver, wait, li_css, timeout=20)

    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", li)
    time.sleep(0.15)

    try:
        wait_clickable(driver, wait, li_css, timeout=10).click()
    except:
        js_click(driver, li)

    # ✅ entryIframe 로딩까지 대기
    switch_default(driver)
    WebDriverWait(driver, 20).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe"))
    )
    # entryIframe 안에서 app-root가 뜰 때까지 대기 (렌더 완료 신호)
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root"))
    )


def extract_detail(driver, wait) -> dict:
    """
    entryIframe(상세)에서:
    - 가게명
    - 방문/블로그리뷰 수
    - 주소(아이콘 클릭 후 텍스트)
    """
    # entryIframe 보장
    switch_to_iframe(driver, wait, "entryIframe")
    wait_presence(driver, wait, "#app-root", timeout=20)

    # 가게명 후보
    name = ""
    for sel in ["h1", "h2", "span.Fc1rA"]:
        t = safe_text(driver, sel)
        if t:
            name = t
            break

    counts = extract_visit_blog_counts(driver)
    address = extract_address(driver, wait)

    return {"name": name, "address": address, **counts}


def get_next_pagination_button(driver, wait):
    """
    searchIframe에서 다음 페이지로 갈 버튼 후보를 찾음.
    """
    switch_to_iframe(driver, wait, "searchIframe")

    try:
        wait_presence(driver, wait, PAGINATION_WRAP, timeout=5)
    except TimeoutException:
        return None

    anchors = driver.find_elements(By.CSS_SELECTOR, PAGINATION_ANCHORS)
    if not anchors:
        return None

    # 오른쪽부터 클릭 가능한 요소 탐색
    for a in reversed(anchors):
        try:
            if not a.is_displayed():
                continue
            aria_disabled = (a.get_attribute("aria-disabled") or "").lower()
            if aria_disabled in ("true", "disabled"):
                continue
            cls = (a.get_attribute("class") or "").lower()
            if "disabled" in cls:
                continue
            return a
        except StaleElementReferenceException:
            continue

    return None


def click_next_page(driver, wait) -> bool:
    """
    다음 페이지 이동. 성공 True / 더 없으면 False
    """
    switch_to_iframe(driver, wait, "searchIframe")

    # 페이지 전환 감지용: 첫 항목 텍스트
    first_before = ""
    try:
        first_before = driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li").text.strip()
    except:
        pass

    btn = get_next_pagination_button(driver, wait)
    if btn is None:
        return False

    try:
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
        time.sleep(0.12)
        btn.click()
    except:
        try:
            js_click(driver, btn)
        except:
            return False

    # ✅ 새 페이지 리스트가 뜰 때까지 기다리기(속도 문제 해결 핵심)
    switch_to_iframe(driver, wait, "searchIframe")
    try:
        WebDriverWait(driver, 15).until(
            lambda d: (d.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container ul > li").text.strip() != first_before)
            if first_before else True
        )
    except:
        pass

    # 첫 li가 존재할 때까지 한 번 더 보장
    try:
        wait_presence(driver, wait, "#_pcmap_list_scroll_container ul > li", timeout=15)
    except:
        return False

    return True


def crawl_all_pages(keyword: str, headless: bool = False, max_pages: int = 999) -> pd.DataFrame:
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    if headless:
        options.add_argument("--headless=new")

    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 20)
    driver.get(URL)

    search_keyword(driver, wait, keyword)

    rows = []
    global_rank = 1
    page_no = 1

    while page_no <= max_pages:
        total = load_all_results_current_page(driver, wait)
        print(f"\n📄 Page {page_no} | 로딩된 결과: {total}개")

        for i in range(1, total + 1):
            # ✅ 항목별 리트라이(중간 멈춤 방지)
            for attempt in range(1, 4):
                try:
                    click_nth_result(driver, wait, i)
                    data = extract_detail(driver, wait)

                    data["page"] = page_no
                    data["rank_in_page"] = i
                    data["rank_global"] = global_rank
                    rows.append(data)

                    print(
                        f"[P{page_no} {i}/{total}] {data['name']} | "
                        f"방문 {data['visit_review_count']} | 블로그 {data['blog_review_count']} | 주소 {data['address']}"
                    )
                    global_rank += 1
                    break  # success
                except Exception as e:
                    if attempt == 3:
                        print(f"[P{page_no} {i}/{total}] ❌ 최종 실패: {e}")
                    else:
                        # 실패 시 잠깐 쉬고 다시 시도
                        time.sleep(0.8)
                        continue

        moved = click_next_page(driver, wait)
        if not moved:
            print("\n✅ 더 이상 다음 페이지가 없어 종료합니다.")
            break

        page_no += 1
        time.sleep(0.8)  # ✅ 페이지 전환 후 안정화

    driver.quit()

    df = pd.DataFrame(rows, columns=[
        "rank_global", "page", "rank_in_page",
        "name", "address",
        "visit_review_count", "blog_review_count",
    ])
    return df


if __name__ == "__main__":
    kw = input("🔍 검색어 입력: ").strip()
    df = crawl_all_pages(kw, headless=False)

    print("\n--- 미리보기 ---")
    print(df.head(20))
    print(f"\n총 수집: {len(df)}건")


🔍 검색어 입력:  중랑구 묵동 일식



📄 Page 1 | 로딩된 결과: 50개
[P1 1/50] 횟집갈래 | 방문 577 | 블로그 150 | 주소 
[P1 2/50] 육연식당 묵동본점 | 방문 1061 | 블로그 77 | 주소 
[P1 3/50] 채선당 묵동점 | 방문 1136 | 블로그 102 | 주소 도로명서울 중랑구 동일로 942 위더스빌 1층 채선당 묵동점복사
지번서울 중랑구 묵동 169-11 위더스빌 1층 채선당 점복사
안내 레이어 닫기
[P1 4/50] 백소정 먹골점 | 방문 4667 | 블로그 147 | 주소 
[P1 5/50] 검색 | 방문 0 | 블로그 0 | 주소 
[P1 6/50]  | 방문 0 | 블로그 0 | 주소 
[P1 7/50] ❌ 최종 실패: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=143.0.7499.192)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6091888d5
	0x7ff609188930
	0x7ff608f6165d
	0x7ff608f391b1
	0x7ff608fe97e6
	0x7ff60900a5c2
	0x7ff608faac29
	0x7ff608faba93
	0x7ff6094a0620
	0x7ff60949af60
	0x7ff6094b96c6
	0x7ff6091a5dd4
	0x7ff6091aed7c
	0x7ff609191ff4
	0x7ff6091921a5
	0x7ff609177ed2
	0x7ffdba15e8d7
	0x7ffdbaa0c53c

[P1 8/50] ❌ 최종 실패: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=143.0.7499.192)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=143.0.7499.192)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6091888d5
	0x7ff609188930
	0x7ff608f6165d
	0x7ff608f391b1
	0x7ff608fe97e6
	0x7ff60900a5c2
	0x7ff608faac29
	0x7ff608faba93
	0x7ff6094a0620
	0x7ff60949af60
	0x7ff6094b96c6
	0x7ff6091a5dd4
	0x7ff6091aed7c
	0x7ff609191ff4
	0x7ff6091921a5
	0x7ff609177ed2
	0x7ffdba15e8d7
	0x7ffdbaa0c53c


### 1

### 1

### 1

In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.by import By

url = "https://map.naver.com/p/search/%EC%A4%91%EB%9E%91%EA%B5%AC%20%EC%83%81%EB%B4%89%EB%8F%99%20%EC%9D%BC%EC%8B%9D/place/1274757035?c=15.00,0,0,0,dh&placePath=/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202601081342&locale=ko&svcName=map_pcv5&searchText=%EC%A4%91%EB%9E%91%EA%B5%AC%20%EC%83%81%EB%B4%89%EB%8F%99%20%EC%9D%BC%EC%8B%9D"

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

driver.get(url)

# 🔹 iframe 진입
driver.switch_to.default_content()
wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

print("✅ entryIframe 진입 완료")

# 여기서부터 driver는 entryIframe 내부 상태

## 상호명 업종명 상호명 업종명 상호명 업종명 상호명 업종명 상호명 업종명 상호명 업종명 
time.sleep(2)
name = driver.find_elements(By.CSS_SELECTOR, "#_title div span.GHAhO")
print("상호명:", name[0].text)
category = driver.find_elements(By.CSS_SELECTOR, "#_title > div > span.lnJFt")
print("업종명:", category[0].text)

add_btn = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div:nth-child(6) div div:nth-child(2) > div.place_section_content > div > div.O8qbU.tQY7D > div a")
add_btn.click()
time.sleep(2)

## 주소 주소 주소 주소 주소 주소 주소 주소 주소 주소 주소 주소 주소 주소 주소

add1 = driver.find_elements(
    By.CSS_SELECTOR,
    "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) "
    "> div.place_section_content > div > div.O8qbU.tQY7D "
    "> div > div.Y31Sf > div:nth-child(1)"
)
print("주소1:", add1[0].text)
add2 = driver.find_elements(
    By.CSS_SELECTOR,
    "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) "
    "> div.place_section_content > div > div.O8qbU.tQY7D "
    "> div > div.Y31Sf > div:nth-child(2)"
)
print("주소2:", add2[0].text)


# 리뷰버튼 리뷰버튼 리뷰버튼 리뷰버튼 리뷰버튼 리뷰버튼 리뷰버튼 리뷰버튼 리뷰버튼 리뷰버튼 리뷰버튼 리뷰버튼 리뷰버튼 


review_btn = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(4) > span")
review_btn.click()
time.sleep(2)


# 스크롤 스크롤 스크롤 스크롤 스크롤 스크롤 스크롤 스크롤 스크롤 스크롤 스크롤 스크롤 스크롤 스크롤



✅ entryIframe 진입 완료
상호명: 와라우 상봉본점
업종명: 이자카야
주소1: 도로명서울 중랑구 봉우재로33길 81 1층 와라우복사
주소2: 지번서울 중랑구 상봉동 89-11복사


In [65]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import (
    NoSuchElementException, StaleElementReferenceException,
    ElementClickInterceptedException, TimeoutException
)


url = "https://map.naver.com/p/search/%EC%A4%91%EB%9E%91%EA%B5%AC%20%EC%83%81%EB%B4%89%EB%8F%99%20%EC%9D%BC%EC%8B%9D/place/1274757035?c=15.00,0,0,0,dh&placePath=/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202601081342&locale=ko&svcName=map_pcv5&searchText=%EC%A4%91%EB%9E%91%EA%B5%AC%20%EC%83%81%EB%B4%89%EB%8F%99%20%EC%9D%BC%EC%8B%9D"

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

driver.get(url)

# 🔹 iframe 진입
driver.switch_to.default_content()
wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

print("✅Iframe 진입 완료")

review_btn = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(6)")
review_btn.click()
time.sleep(2)


####################  리뷰더보기 버튼 리뷰더보기 버튼 리뷰더보기 버튼 리뷰더보기 버튼 리뷰더보기 버튼 리뷰더보기 버튼 리뷰더보기 버튼 

MORE_BTN_SELECTOR = (
    "#app-root > div > div > div:nth-child(7) > div:nth-child(3) "
    "> div.place_section.k1QQ5 > div.NSTUp > div > a > span"
)

for _ in range(4):
    driver.find_element(By.CSS_SELECTOR, MORE_BTN_SELECTOR).click()
    time.sleep(2)

################# 


############### 리뷰 가져오기 리뷰 가져오기 리뷰 가져오기 리뷰 가져오기 리뷰 가져오기 리뷰 가져오기 



REVIEW_LI = "#_review_list > li"
REVIEW_WRAP = "div.pui__vn15t2"                 # 리뷰 본문 영역
READ_MORE_CANDIDATES = [
    "a.pui__jhpEy",                              # 더보기 버튼에서 자주 보이는 클래스
    "a[role='button']",
    "a"                                          # 마지막 fallback: a들 중 '더보기' 텍스트 찾기
]

def js_click(driver, el):
    driver.execute_script("arguments[0].click();", el)

def scroll_under_header(driver, el):
    # 헤더에 가리지 않게 start로 올린 뒤 헤더 높이만큼 조금 더 내림
    driver.execute_script("arguments[0].scrollIntoView({block:'start'});", el)
    time.sleep(0.15)
    header_h = driver.execute_script("""
        const h = document.querySelector('#_header');
        return h ? h.getBoundingClientRect().height : 0;
    """)
    driver.execute_script("window.scrollBy(0, arguments[0] + 12);", header_h)
    time.sleep(0.15)

def click_read_more_if_exists(driver, wrap):
    """
    wrap(div.pui__vn15t2) 안에서 '더보기'로 보이는 링크가 있으면 클릭.
    없으면 그냥 넘어감.
    """
    # 후보 셀렉터들을 돌면서 a들을 모아 '더보기' 텍스트 매칭
    for sel in READ_MORE_CANDIDATES:
        try:
            anchors = wrap.find_elements(By.CSS_SELECTOR, sel)
        except Exception:
            anchors = []

        for a in anchors:
            try:
                txt = (a.text or "").strip()
                if "더보기" in txt:
                    scroll_under_header(driver, a)
                    try:
                        a.click()
                    except ElementClickInterceptedException:
                        js_click(driver, a)
                    time.sleep(0.2)
                    return True
            except StaleElementReferenceException:
                continue
    return False

def extract_review_text_from_wrap(wrap):
    """
    wrap 내부에서 실제 리뷰 텍스트를 최대한 안전하게 가져오기.
    - '더보기' 링크 텍스트는 제외(대부분 a.text로는 '더보기'만 잡히는 경우가 있어)
    - wrap.text에서 '더보기' 단어 제거
    """
    raw = (wrap.text or "").strip()
    raw = raw.replace("더보기", "").strip()
    return raw

def crawl_visible_reviews(driver, wait_sec=10):
    wait = WebDriverWait(driver, wait_sec)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, REVIEW_LI)))

    reviews = []
    li_elems = driver.find_elements(By.CSS_SELECTOR, REVIEW_LI)

    for idx in range(len(li_elems)):
        # DOM이 바뀌는 경우 대비: 매번 새로 li를 다시 잡아줌
        for _ in range(3):
            try:
                li = driver.find_elements(By.CSS_SELECTOR, REVIEW_LI)[idx]
                wrap = li.find_element(By.CSS_SELECTOR, REVIEW_WRAP)

                # (있으면) 더보기 클릭해서 펼치기
                click_read_more_if_exists(driver, wrap)

                # 펼친 뒤 텍스트 추출
                text = extract_review_text_from_wrap(wrap)
                if text:
                    reviews.append(text)
                break
            except (StaleElementReferenceException, NoSuchElementException):
                time.sleep(0.2)
                continue

    return reviews

# ---- 사용 예시 ----
# 리뷰 탭 진입 + 리스트 로딩 이후에 호출
reviews = crawl_visible_reviews(driver)
print("수집 리뷰 수:", len(reviews))
print(reviews[:])


#############################################

✅Iframe 진입 완료
수집 리뷰 수: 50
['저번에 방문하고 재방문입니다!\n안주가 맛있었던 기억이 있어서 다른 친구들이랑 또 갔습니다~\n화장실 가게 안에 있어서 넘 편하고\n피규어가 많아서 친구들이 좋아했어요!\n그리고 빔으로 진격의 거인을 틀어놔 주시더라구요\n신죠오사사게오 외치고 안주 먹고왔습니다 ~\n접기', '흑 크리스마스때 일해서 못왔는데 오늘 왔어요!@!\n단골최애집 맛도리입니다ㅠㅠㅠ 오늘은 좀 기다렸다 들어와쑨ㄴ데,, 여기 맛도있고 사장님들도 너므ㅜ친절하세여', '[중랑] 와라우 상봉본점\n\n✔️ 7호선, 경의중앙, 경춘선 상봉역 2번 출구에서 459m\n✔️ 정통 일본식 이자카야 스타일을 현대적으로 재해석한 요리와 술을 제공\n✔️ 다양한 일본 술과 독창적인 요리의 완벽한 조화를 선보이는 이자카야\n\n✨ 사시미류 - 육회+연어 / 연어사시미 / 한우투뿔육회 / 한우물회 / 딱새우회 / 소고기타다끼\n✨ 철판,볶음류 - 치킨커리파스타 / 카니미소오일파스타 / 새우감바스 / 닭목살볶음 / 오코노미야끼\n✨ 튀김류 - 유린기 / 육즙가지롤 / 치킨가라아게 / 닭날개튀김 / 게소아게 / 새우깡\n✨ 나베류 / 구이류 / 꼬치류 / 식사류 등\n접기', '와우..간신히 웨이팅 없이 바로 입장해서 행운이었어요~\n네이버로 검색후 찜한 장소였는데 생각보다 넘 만족한 곳이었이요~~다음엔 2차 때말고 바로 약속 장소로 픽!!!!안주도 깨끗 깔끔 제맘레 쏙 들었습니다~~\n접기', '원래 좋아하는 상봉 술집 와라우! 만석으로 살짝 웨이팅 하였지만 역시 그 가치가 있네요✌️\n\n다들 와라우 왜 안가세요ㅠㅠ 꼬치는 물론이고 특히 닭목살 미쳤어여... 저것만 먹으러 당장 달려가구 싶을 정도랍니다❤️\n\n나중에 또 와라우로 달려오겠습니다ㅠㅠㅠㅠ 와라우 싸랑해용ㅠㅠㅠㅠㅠㅠ\n접기', '와우~ 입구부터 제 취향이었어요. 피큐어보고 심장이 멎을뻔 했습니다😍 음식 비쥬얼도 장난아니구요. 데이트장소로 최고에요😜', '음식 너무 맛있고 분위기도 좋아요🥰\n친구랑 수

In [ ]:
# TAG_NAME : 태그명으로 접근
body = driver.find_element(By.TAG_NAME, 'body')
# (By.CSS_SELECTOR, 'body.wrap-new.api_animation')  로 사용해도 됨

# END키를 누르면 현재 페이지의 스크롤을 가장 밑으로 내려줌
body.send_keys(Keys.END)    # DOWN은 아래 방향 키보드 한번 클릭